In [8]:
import os
import pandas as pd
from my_utils import stardisting as sd
from tqdm import tqdm
from my_utils import published_data_processing as pub
from my_utils import tile_processing as tp

In [9]:
path_20x_public_downsample = "Z:\Interpolations\L - 20x - 1_"
full_dataset = pub.read_all_20x_published_data(path_20x_public_downsample)

In [10]:
path_models_sd = r"Z:\Segmentation Models\StarDist"
inputs_csv = "SD_HE_20x Candidate Inputs.csv"
df = pd.read_csv(os.path.join(path_models_sd, inputs_csv))
df

,Model Name Out,Initial Model,Random Split State,Epochs,Learning Rate,Aug Rot90,Aug Flip,Aug Intensity,Aug Hue,Aug Blur
0,Model _01,2D_versatile_he,7,50,0.00010,False,False,False,False,False
1,Model _02,Model _01,7,50,0.00010,False,False,False,False,False
2,Model _03,Model _02,7,50,0.00010,False,False,False,False,False
3,Model _04,2D_versatile_he,7,50,0.00001,False,False,False,False,False
4,Model _05,Model _04,7,50,0.00001,False,False,False,False,False
5,Model _06,Model _05,7,50,0.00001,False,False,False,False,False


In [11]:
for _, row in tqdm(df.iterrows()):
    model_name, starting_model, r_state, epochs, lr, rot90, flip, intensity, hue, blur = [None for _ in range(len(row))]
    for item, value in row.items():
        if item == 'Model Name Out':
            model_name = value
        elif item == 'Initial Model':
            starting_model = value
        elif item == 'Random Split State':
            r_state = value
        elif item == 'Epochs':
            epochs = value
        elif item == 'Learning Rate':
            lr = value
        elif item == 'Aug Rot90':
            rot90 = value
        elif item == 'Aug Flip':
            flip = value
        elif item == 'Aug Intensity':
            intensity = value
        elif item == 'Aug Hue':
            hue = value
        elif item == 'Aug Noise':
            noise = value   
        elif item == 'Aug Blur':
            blur = value
    if starting_model == '2D_versatile_he':
        model = sd.load_published_he_model(path_models_sd, model_name)
    elif starting_model == 'Random Initialization':
        model = sd.load_random_he_model(path_models_sd, model_name)
    else:
        model = sd.load_model(model_path=os.path.join(path_models_sd, starting_model), new_model_path_for_retraining=os.path.join(path_models_sd, model_name))

    trn_vld_tst = pub.split_all_data([75, 15, 10], full_dataset, r_state)
    trn = {'Images': [], 'Masks': []}
    vld = {'Images': [], 'Masks': []}
    for dataset in list(full_dataset.keys()):
        [trn['Images'].append(trn_vld_tst['Train'][dataset]['Images'][i]) for i, img in enumerate(trn_vld_tst['Train'][dataset]['Images'])]
        [trn['Masks'].append(trn_vld_tst['Train'][dataset]['Masks'][i]) for i, msk in enumerate(trn_vld_tst['Train'][dataset]['Masks'])]
        [vld['Images'].append(trn_vld_tst['Validate'][dataset]['Images'][i]) for i, img in enumerate(trn_vld_tst['Validate'][dataset]['Images'])]
        [vld['Masks'].append(trn_vld_tst['Validate'][dataset]['Masks'][i]) for i, msk in enumerate(trn_vld_tst['Validate'][dataset]['Masks'])]
    
    if rot90 or flip or intensity or hue or blur:
        aug_images, aug_masks = [], []
        for i, img in enumerate(trn['Images']):
            msk = trn['Masks'][i]
            augmenter = tp.TilePairAugmenter(img, msk, random_state=i, rotate90=rot90, flip=flip, intensity=intensity, hue=hue, blur=blur)
            aug_images.append(augmenter.image_rgb)
            aug_masks.append(augmenter.mask_gray)
        [trn['Images'].append(img) for img in aug_images]
        [trn['Masks'].append(msk) for msk in aug_masks]
    
    model = sd.configure_model_for_training(model=model, use_gpu=True, epochs=epochs, learning_rate=lr)
    model = sd.normalize_train_and_threshold(model=model, training_images=trn['Images'], training_masks=trn['Masks'],
                                         validation_images=vld['Images'], validation_masks=vld['Masks'])
    

0it [00:00, ?it/s]base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\Segmentation Models\StarDist\Model _01


Found model '2D_versatile_he' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.692478, nms_thresh=0.3.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.619771, nms_thresh=0.3.

Ignore that, thresholds are: Thresholds(prob=0.6924782541382084, nms=0.3)
Epoch 1/50
100/100 [==============================] - 7s 57ms/step - loss: 0.5398 - prob_loss: 0.2145 - dist_loss: 1.6263 - prob_kld: 0.0966 - dist_relevant_mae: 1.6260 - dist_relevant_mse: 6.4237 - dist_dist_iou_metric: 0.5748 - val_loss: 0.5164 - val_prob_loss: 0.2226 - val_dist_loss: 1.4692 - val_prob_kld: 0.0930 - val_dist_relevant_mae: 1.4689 - val_dist_relevant_mse: 5.3593 - val_dist_dist_iou_metric: 0.6259 - lr: 1.0000e-04
Epoch 2/50
100/100 [==============================] - 4s 41ms/step - loss: 0.5096 - prob_loss: 0.2058 - dist_loss: 1.5192 - prob_kld: 0.0881 - dist_relevant_mae: 1.5189 - dist_relevant_mse:


NMS threshold = 0.3:  75%|███████▌  | 15/20 [00:13<00:04,  1.10it/s, 0.693 -> 0.545]

NMS threshold = 0.4:  75%|███████▌  | 15/20 [00:12<00:04,  1.18it/s, 0.693 -> 0.545]

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:12<00:04,  1.16it/s, 0.693 -> 0.543]
1it [03:59, 239.70s/it]base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\Segmentation Models\StarDist\Model _02


Using optimized values: prob_thresh=0.690146, nms_thresh=0.4.
Saving to 'thresholds.json'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.510036, nms_thresh=0.3.
Overriding defaults: Thresholds(prob=0.6901457253152334, nms=0.4) 
Epoch 1/50
100/100 [==============================] - 6s 52ms/step - loss: 0.3887 - prob_loss: 0.1711 - dist_loss: 1.0877 - prob_kld: 0.0558 - dist_relevant_mae: 1.0874 - dist_relevant_mse: 3.0908 - dist_dist_iou_metric: 0.6803 - val_loss: 0.4866 - val_prob_loss: 0.2212 - val_dist_loss: 1.3269 - val_prob_kld: 0.0934 - val_dist_relevant_mae: 1.3266 - val_dist_relevant_mse: 4.8079 - val_dist_dist_iou_metric: 0.6385 - lr: 1.0000e-04
Epoch 2/50
100/100 [==============================] - 4s 39ms/step - loss: 0.3917 - prob_loss: 0.1735 - dist_loss: 1.0910 - prob_kld: 0.0558 - dist_relevant_mae: 1.0908 - dist_relevant_mse: 3.1454 - dist_dist_iou_metric: 0.6798 - val_loss: 0.4913 - val_prob_loss: 0.2212 - val_dist_loss: 1.3503 - val_prob


NMS threshold = 0.3:  75%|███████▌  | 15/20 [00:12<00:04,  1.19it/s, 0.676 -> 0.540]

NMS threshold = 0.4:  75%|███████▌  | 15/20 [00:12<00:04,  1.16it/s, 0.676 -> 0.539]

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:12<00:04,  1.21it/s, 0.693 -> 0.538]
2it [07:57, 238.87s/it]base_model.py (149): output path for model already exists, files may be overwritten: \\10.99.68.53\Digital pathology image lib\_Image libraries for training\2023-05-09 Published HE Nuclei Datasets\Segmentation Models\StarDist\Model _03


...

Using optimized values: prob_thresh=0.515186, nms_thresh=0.3.
Saving to 'thresholds.json'.
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.5151855589946364, nms=0.3) 
Epoch 1/50
100/100 [==============================] - 6s 48ms/step - loss: 0.4746 - prob_loss: 0.1947 - dist_loss: 1.3994 - prob_kld: 0.0772 - dist_relevant_mae: 1.3991 - dist_relevant_mse: 4.8873 - dist_dist_iou_metric: 0.6169 - val_loss: 0.4901 - val_prob_loss: 0.2173 - val_dist_loss: 1.3641 - val_prob_kld: 0.0898 - val_dist_relevant_mae: 1.3638 - val_dist_relevant_mse: 4.7282 - val_dist_dist_iou_metric: 0.6203 - lr: 1.0000e-05
Epoch 2/50
100/100 [==============================] - 4s 39ms/step - loss: 0.4741 - prob_loss: 0.1962 - dist_loss: 1.3892 - prob_kld: 0.0784 - dist_relevant_mae: 1.3889 - dist_relevant_mse: 4.8265 - dist_dist_iou_metric: 0.6193 - val_loss: 0.4923 - val_prob_loss: 0.2174 - val_dist_loss: 1.3743 - val_prob_kld: 0.0899 - val_dist_relevant_mae: 1.3739 - v


NMS threshold = 0.3:  75%|███████▌  | 15/20 [00:11<00:03,  1.28it/s, 0.581 -> 0.525]

NMS threshold = 0.4:  75%|███████▌  | 15/20 [00:11<00:03,  1.26it/s, 0.556 -> 0.524]

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:12<00:04,  1.19it/s, 0.597 -> 0.522]
5it [19:41, 234.89s/it]

Using optimized values: prob_thresh=0.578151, nms_thresh=0.3.
Saving to 'thresholds.json'.
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Overriding defaults: Thresholds(prob=0.5781513259405411, nms=0.3) 
Epoch 1/50
100/100 [==============================] - 6s 48ms/step - loss: 0.4592 - prob_loss: 0.1926 - dist_loss: 1.3331 - prob_kld: 0.0736 - dist_relevant_mae: 1.3328 - dist_relevant_mse: 4.4981 - dist_dist_iou_metric: 0.6289 - val_loss: 0.4839 - val_prob_loss: 0.2113 - val_dist_loss: 1.3628 - val_prob_kld: 0.0844 - val_dist_relevant_mae: 1.3624 - val_dist_relevant_mse: 4.9269 - val_dist_dist_iou_metric: 0.6341 - lr: 1.0000e-05
Epoch 2/50
100/100 [==============================] - 4s 38ms/step - loss: 0.4593 - prob_loss: 0.1920 - dist_loss: 1.3367 - prob_kld: 0.0738 - dist_relevant_mae: 1.3364 - dist_relevant_mse: 4.5203 - dist_dist_iou_metric: 0.6275 - val_loss: 0.4850 - val_prob_loss: 0.2114 - val_dist_loss: 1.3683 - val_prob_kld: 0.0844 - val_dist_relevant_mae: 1.3680 - v


NMS threshold = 0.3:  75%|███████▌  | 15/20 [00:13<00:04,  1.13it/s, 0.551 -> 0.529]

NMS threshold = 0.4:  75%|███████▌  | 15/20 [00:12<00:04,  1.19it/s, 0.619 -> 0.528]

NMS threshold = 0.5:  75%|███████▌  | 15/20 [00:12<00:04,  1.21it/s, 0.630 -> 0.526]
6it [23:32, 235.45s/it]

Using optimized values: prob_thresh=0.553049, nms_thresh=0.3.
Saving to 'thresholds.json'.
